# Make new csv with relabelled data

Reads in a results.csv and create a full dataset with extra columns
Oct 21, 2020

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import subprocess as sp
import pickle
#import ipywidgets as widgets
import time

import pandas as pd

## Read data

In [14]:
fname='/global/cfs/cdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/autoscan_features.3.csv'
df_full=pd.read_csv(fname,sep=',',comment='#')
df_full.head()

,ID,OBJECT_TYPE,AMP,A_IMAGE,A_REF,BAND,B_IMAGE,B_REF,CCDID,COLMEDS,...,N3SIG3,N3SIG3SHIFT,N3SIG5,N3SIG5SHIFT,NN_DIST_RENORM,NUMNEGRN,SCALE,SNR,SPREADERR_MODEL,SPREAD_MODEL
0,10742010,0,0.808323,1.508,2.650069,i,0.950,1.899501,10,0.112077,...,0,-8,0,-9,0.674934,22,2.024122,7.722346,0.004629,-0.003718
1,8828139,0,0.975141,1.430,NaN,i,1.319,NaN,10,0.072256,...,0,5,0,5,NaN,21,1.379283,25.582918,0.002420,-0.000578
2,8229889,0,0.877440,0.505,NaN,z,0.494,NaN,57,0.106124,...,0,1,0,1,NaN,20,0.950069,3.914192,0.008863,-0.020044
3,11033218,0,0.681179,0.762,3.008957,i,0.449,2.855881,1,0.139071,...,0,-14,2,-27,1.240574,18,1.749488,5.068382,0.008041,0.000246
4,8341858,0,0.645906,1.205,1.352224,i,0.982,1.063333,30,0.123302,...,0,4,0,4,2.970234,16,1.930613,7.372579,0.005606,0.001771


In [15]:
df2=pd.read_csv('fulldataset_relabeling_results.csv')
display(df2.head())
print(df2[df2.old_label!=df2.final_label].shape,df2.shape)

,ID,old_label,changed_label,final_label
0,8809567,1,0,1
1,8811073,1,0,0
2,7012253,1,0,1
3,10699160,1,0,0
4,9237522,1,0,0


## Add columns and insert new labels

In [17]:
## Add extra columns
df_full['old_OBJECT_TYPE']=df_full['OBJECT_TYPE']
df_full['check_relabel']=np.zeros(df_full.shape[0],dtype=bool) ## Whether checked for relabelling. Not necessarily modified
print(df_full.shape)

(898963, 42)


In [18]:
id_list=df2.ID.values

df=df_full.copy()
for i,j in df[df.ID.isin(id_list)].iterrows():
#     print(i,j.relabelled)
    iD=j.ID
    new_label=df2[df2.ID==iD]['final_label'].values[0]
    df.loc[i,'check_relabel']=True
    df.loc[i,'OBJECT_TYPE']=new_label
    

In [51]:
# df[df.old_OBJECT_TYPE!=df.OBJECT_TYPE]

In [47]:
## Save to new csv file
op_fname='relabeled_autoscan_features.csv'
# df.to_csv('/global/cfs/cdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/'+op_fname,index=False)

## Create new .npy file with relabelled labels

In [9]:
# Read stored IDs and old labels
main_dir='/global/cfs/cdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/input_npy_files/'
IDs=np.load(main_dir+'full_idx.npy')
old_labels=np.load(main_dir+'old_full_y.npy')

In [38]:
# Re-arrange DataFrame according to the order of the IDs used in full_idx.npy
df3=df.set_index('ID',drop=False)
df3=df3.reindex(IDs) # Set order according to IDs
display(df3.head())

## Check ID and old labels order
print('ID order:',np.array_equal(df3.ID.values,IDs))
print('old labels order:',np.array_equal(df3.old_OBJECT_TYPE.values,old_labels)) 

,ID,OBJECT_TYPE,AMP,A_IMAGE,A_REF,BAND,B_IMAGE,B_REF,CCDID,COLMEDS,...,N3SIG5,N3SIG5SHIFT,NN_DIST_RENORM,NUMNEGRN,SCALE,SNR,SPREADERR_MODEL,SPREAD_MODEL,old_OBJECT_TYPE,check_relabel
ID,,,,,,,,,,,,,,,,,,,,,
7865232,7865232,1,0.551049,1.088,2.964917,g,0.825,2.741324,12,0.070000,...,5,-41,0.966962,21,1.740246,5.563241,0.006092,-0.013196,1,False
8633234,8633234,1,0.650608,0.577,4.680729,g,0.289,2.786566,10,0.055429,...,5,-42,0.697808,24,1.249333,3.763624,0.009706,-0.016714,1,False
11033574,11033574,1,0.716091,0.846,2.417334,i,0.668,2.360914,38,0.099660,...,1,-26,0.912621,18,1.174316,6.169670,0.007717,-0.020928,1,False
9287247,9287247,1,0.314339,2.497,5.044860,i,1.228,4.323637,38,0.041800,...,5,-42,1.056528,13,2.026972,7.314532,0.005306,0.004191,1,False
11470636,11470636,0,0.964118,2.124,4.503510,r,1.962,4.441669,55,0.023034,...,0,-29,0.803004,5,1.832286,54.761597,0.000637,-0.000308,0,False


ID order: True
old labels order: True


In [45]:
## Save new labels
new_labels=df3.OBJECT_TYPE.values
# np.save(main_dir+'new_relabeled_full_y.npy',new_labels)